In [15]:
import torch
import torch.nn as nn
import torch.utils.data as data_utils
from torchsummary import summary
from torchvision import models
import pytorch_lightning as pl
import wandb

import os, sys
sys.path.append(os.path.abspath("../"))

%load_ext autoreload
%autoreload 2

from models.autoencoders.ae import ResUnetAE
from models.autoencoders.components import UpBlock

from datasets.coco.datasets import CocoDataLoader
from datasets.coco.dataloaders import singles_dataloader

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data

In [3]:
cocoDataLoader = CocoDataLoader()

loading annotations into memory...
Done (t=99.28s)
creating index...
index created!


In [4]:
dataloaders = singles_dataloader(
    cocoDataLoader.singles_dataset(slice(0, 12)),
    2/3, 4, True, 0
)

In [5]:
b = next(iter(dataloaders["train"]))

In [6]:
b

[tensor([[[[ 0.0505,  0.1276,  0.0441,  ..., -0.1892, -0.4696, -0.1849],
           [-0.0073,  0.0098,  0.1297,  ..., -0.3048, -0.3455, -0.3412],
           [ 0.1147,  0.0291,  0.1854,  ..., -0.3755, -0.2641, -0.0779],
           ...,
           [ 0.3673,  0.4637,  0.9496,  ...,  0.8918,  0.7826,  0.7184],
           [ 0.6649,  0.9431,  1.1465,  ..., -0.2235,  0.0313,  0.9196],
           [ 0.8618,  0.9260,  0.6370,  ...,  0.7719,  0.7505,  0.2004]],
 
          [[ 0.2599,  0.4109,  0.1045,  ..., -0.0749, -0.2872, -0.1690],
           [ 0.0367,  0.0783,  0.2074,  ..., -0.2084, -0.3069, -0.1078],
           [ 0.2293,  0.3080,  0.1767,  ..., -0.1318, -0.2062,  0.0192],
           ...,
           [ 0.6057,  0.8704,  1.2228,  ...,  1.1878,  1.0696,  0.9821],
           [ 0.8792,  1.2250,  1.3672,  ...,  0.2139,  0.3474,  1.1243],
           [ 1.0871,  1.1856,  0.8858,  ...,  1.0827,  1.0018,  0.6166]],
 
          [[-0.4341, -0.4842, -0.4711,  ..., -0.5278, -0.4166, -0.5038],
           [-

## Network

In [7]:
resnet = models.resnet18(pretrained=True)

In [8]:
bridge_out_channels = 1024

In [9]:
autoencoder = ResUnetAE(
    resnet,
    bridge_out_channels,
    UpBlock.UPSAMPLING_BILINEAR,
    (128, 128), 
    lr=0.02
)

In [10]:
autoencoder(torch.rand(1, 3, 232, 232)).shape

torch.Size([1, 1024, 4, 4])

In [19]:
wandb.login()
wandb_logger = pl.loggers.WandbLogger()
trainer = pl.Trainer(logger=wandb_logger, max_epochs=5)
trainer.fit(autoencoder, dataloaders["train"])
wandb.finish()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
/usr/local/anaconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:101: UserWarning: you defined a validation_step but have no val_dataloader. Skipping val loop
  rank_zero_warn(f'you defined a {step_name} but have no {loader_name}. Skipping {stage} loop')
wandb: W&B syncing is set to `offline` in this directory.  Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.

  | Name    | Type          | Params
------------------------------------------
0 | encoder | ResnetEncoder | 11.2 M
1 | maxpool | MaxPool2d     | 0     
2 | bridge  | Bridge        | 14.2 M
3 | decoder | UnetDecoder   | 10.2 M
------------------------------------------
35.6 M    Trainable params
0         Non-trainable params
35.6 M    Total params
142.228   Total estimated model params size (MB)
Epoch 0:   0%|          | 0/2 [00:00<?, ?it/s] /usr/local/anaconda3/lib/python3.8/site-packages/pytorch

wandb: You can sync this run to the cloud by running:
wandb: wandb sync /Users/francoisledoyen/Documents/FAC/GENERATION_TEXTE_IMAGES/code/contrast-generation/notebooks/wandb/offline-run-20210628_151629-k6earcj5
